In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Set up Seaborn for better aesthetics
sns.set_theme(style='darkgrid')
plt.rcParams['figure.facecolor'] = '#2E3440'
plt.rcParams['axes.facecolor'] = '#3B4252'
plt.rcParams['text.color'] = '#E5E9F0'
plt.rcParams['axes.labelcolor'] = '#E5E9F0'
plt.rcParams['xtick.color'] = '#E5E9F0'
plt.rcParams['ytick.color'] = '#E5E9F0'
plt.rcParams['axes.titlecolor'] = '#E5E9F0'
plt.rcParams['legend.facecolor'] = '#3B4252'
plt.rcParams['legend.edgecolor'] = '#5E81AC'
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['legend.title_fontsize'] = 14
plt.rcParams['grid.color'] = '#4C566A'

In [ ]:
def calculate_retirement_data(
    current_age=38,
    min_age_of_death=60,
    max_age_of_death=100,
    current_yearly_income=120_000,
    current_yearly_expenses=50_500,
    current_savings=0,
    savings_growth_rates=[0.03, 0.05, 0.07],  # Multiple scenarios
    inflation_rates=[0.02, 0.03],  # Multiple scenarios
    income_increases=[(5, 1.2), (10, 1.5)],  # (years_from_now, multiplier)
    desired_retirement_expenses=None,  # If None, will use current_yearly_expenses
):
    if desired_retirement_expenses is None:
        desired_retirement_expenses = current_yearly_expenses

    # Create a grid of all scenarios to test
    scenarios = []
    for growth_rate in savings_growth_rates:
        for inflation_rate in inflation_rates:
            scenarios.append(
                {
                    'growth_rate': growth_rate,
                    'inflation_rate': inflation_rate,
                    'label': f'Growth {growth_rate * 100:.1f}%, Inflation {inflation_rate * 100:.1f}%',
                }
            )

    all_results = []

    # Process each scenario
    for scenario in scenarios:
        growth_rate = scenario['growth_rate']
        inflation_rate = scenario['inflation_rate']
        label = scenario['label']

        earliest_retirement_ages = []

        # Iterate through each estimated age of death
        for estimated_age_of_death in range(min_age_of_death, max_age_of_death + 1):
            # Determine the earliest possible retirement age
            found_retirement_age = False
            for potential_retirement_age in range(current_age, estimated_age_of_death):
                years_until_retirement = potential_retirement_age - current_age
                years_of_retirement_expenses = estimated_age_of_death - potential_retirement_age

                # Calculate savings at the proposed retirement age with income increases
                future_savings = current_savings
                yearly_income = current_yearly_income

                for year in range(years_until_retirement):
                    # Apply income increases if we hit specified intervals
                    for years_from_now, multiplier in income_increases:
                        if year == years_from_now:
                            yearly_income *= multiplier

                    # Calculate savings growth
                    future_savings *= 1 + growth_rate
                    future_savings += yearly_income - current_yearly_expenses

                # Calculate the total expenses during retirement, factoring in inflation
                total_expenses = 0
                for year in range(years_of_retirement_expenses):
                    total_expenses += desired_retirement_expenses * ((1 + inflation_rate) ** year)

                # Determine if retirement is possible at this age
                if future_savings >= total_expenses:
                    earliest_retirement_ages.append((estimated_age_of_death, potential_retirement_age))
                    found_retirement_age = True
                    break

            if not found_retirement_age:
                # If no feasible retirement age was found, append "None"
                earliest_retirement_ages.append((estimated_age_of_death, None))

        # Create a DataFrame for this scenario
        df = pd.DataFrame(earliest_retirement_ages, columns=['Planned Age of Death', 'Earliest Retirement Age'])
        df['Scenario'] = label
        df['Growth Rate'] = growth_rate
        df['Inflation Rate'] = inflation_rate
        all_results.append(df)

    # Combine all results
    combined_df = pd.concat(all_results, ignore_index=True)

    # Use the Nord color palette which works great in dark mode
    nord_palette = ['#88C0D0', '#8FBCBB', '#81A1C1', '#5E81AC', '#EBCB8B', '#A3BE8C', '#B48EAD', '#D08770', '#BF616A']

    # Plot the results with subplots grouped by inflation rate
    unique_inflation_rates = sorted(combined_df['Inflation Rate'].unique())
    num_inflation_rates = len(unique_inflation_rates)

    # Create a figure with subplots - one per inflation rate
    fig, axes = plt.subplots(num_inflation_rates, 1, figsize=(24, 7 * num_inflation_rates), sharex=True)
    fig.patch.set_facecolor('#2E3440')  # Dark background for the entire figure

    if num_inflation_rates == 1:
        axes = [axes]  # Make it iterable when there's only one subplot

    # Create each subplot
    for i, inflation_rate in enumerate(unique_inflation_rates):
        ax = axes[i]
        ax.set_facecolor('#3B4252')  # Set subplot background

        inflation_data = combined_df[combined_df['Inflation Rate'] == inflation_rate]

        # Create a temporary DataFrame for seaborn plotting
        plot_data = pd.DataFrame()

        # Plot each growth rate as a different line
        for j, growth_rate in enumerate(sorted(savings_growth_rates)):
            scenario_data = inflation_data[inflation_data['Growth Rate'] == growth_rate].copy()

            # Skip if no valid data
            if scenario_data.empty:
                continue

            # Sort by Planned Age of Death for smooth lines
            scenario_data = scenario_data.sort_values('Planned Age of Death')
            scenario_data['Growth Rate Label'] = f'{growth_rate * 100:.1f}%'

            # Add to plotting DataFrame
            plot_data = pd.concat([plot_data, scenario_data])

        # Use Seaborn for plotting with better aesthetics
        sns.lineplot(
            data=plot_data,
            x='Planned Age of Death',
            y='Earliest Retirement Age',
            hue='Growth Rate Label',
            palette=nord_palette[: len(savings_growth_rates)],
            marker='o',
            markersize=8,
            linewidth=2.5,
            ax=ax,
        )

        # Enhance the subplot
        ax.set_ylabel('Earliest Retirement Age', fontsize=14, fontweight='bold')
        ax.set_title(f'Inflation Rate: {inflation_rate * 100:.1f}%', fontsize=18, fontweight='bold', pad=15)

        # Customize the legend
        legend = ax.legend(title='Growth Rate', frameon=True, fontsize=12, title_fontsize=14)
        legend.get_frame().set_linewidth(2.0)

        # Set y-axis ticks appropriately based on data in this subplot
        valid_ages = inflation_data['Earliest Retirement Age'].dropna()
        if not valid_ages.empty:
            min_retirement_age = max(current_age, valid_ages.min() - 1)
            max_retirement_age = valid_ages.max() + 1
            ax.set_yticks(range(int(min_retirement_age), int(max_retirement_age) + 1, 1))

        # Add a shaded region for the feasible retirement ages
        ax.axhspan(current_age, current_age + 10, alpha=0.15, color='#A3BE8C', zorder=0)
        ax.axhspan(current_age + 10, current_age + 20, alpha=0.1, color='#EBCB8B', zorder=0)

        # Add annotations for context
        ax.text(
            min_age_of_death + 3, current_age + 5, 'Preferred Retirement Zone', fontsize=12, color='#A3BE8C', alpha=0.9
        )

    # Set common x-axis properties
    axes[-1].set_xlabel('Planned Age of Death', fontsize=14, fontweight='bold')
    axes[-1].set_xticks(range(min_age_of_death, max_age_of_death + 1, 5))

    # Add title to the overall figure
    fig.suptitle('Retirement Planning Scenarios', fontsize=24, y=0.98, fontweight='bold', color='#E5E9F0')

    # Display information about income increases
    income_text = 'Income Progression: '
    for years, multiplier in income_increases:
        income_text += f'+{years} years: {(multiplier - 1) * 100:.0f}% increase, '

    fig.text(
        0.5,
        0.01,
        income_text[:-2],
        ha='center',
        fontsize=13,
        color='#88C0D0',
        bbox=dict(facecolor='#434C5E', edgecolor='#5E81AC', alpha=0.7, boxstyle='round,pad=0.5'),
    )

    # Add context information
    context_text = f'Starting Savings: ${current_savings:,}   |   Annual Income: ${current_yearly_income:,}   |   Annual Expenses: ${current_yearly_expenses:,}'
    fig.text(
        0.5,
        0.04,
        context_text,
        ha='center',
        fontsize=13,
        color='#E5E9F0',
        bbox=dict(facecolor='#434C5E', edgecolor='#5E81AC', alpha=0.7, boxstyle='round,pad=0.5'),
    )

    plt.tight_layout(rect=[0, 0.06, 1, 0.95])
    plt.show()

    # Create a combined view with all scenarios
    plt.figure(figsize=(24, 12))
    fig = plt.gcf()
    fig.patch.set_facecolor('#2E3440')  # Dark background for the entire figure
    ax = plt.gca()
    ax.set_facecolor('#3B4252')  # Set axes background

    # Create a single Seaborn plot with all scenarios
    plot_data = pd.DataFrame()

    for i, scenario in enumerate(scenarios):
        growth_rate = scenario['growth_rate']
        inflation_rate = scenario['inflation_rate']
        label = scenario['label']

        scenario_data = combined_df[
            (combined_df['Growth Rate'] == growth_rate) & (combined_df['Inflation Rate'] == inflation_rate)
        ].copy()

        # Skip if no valid data
        if scenario_data.empty:
            continue

        # Sort for smooth lines
        scenario_data = scenario_data.sort_values('Planned Age of Death')

        # Add to plotting DataFrame
        plot_data = pd.concat([plot_data, scenario_data])

    # Use Seaborn for the combined plot
    sns.lineplot(
        data=plot_data,
        x='Planned Age of Death',
        y='Earliest Retirement Age',
        hue='Scenario',
        palette=nord_palette[: len(scenarios)],
        marker='o',
        markersize=8,
        linewidth=2.5,
    )

    # Additional styling
    plt.xlabel('Planned Age of Death', fontsize=16, fontweight='bold')
    plt.ylabel('Earliest Retirement Age', fontsize=16, fontweight='bold')
    plt.title('Retirement Planning Scenarios - All Combinations', fontsize=22, fontweight='bold', pad=20)
    plt.xticks(range(min_age_of_death, max_age_of_death + 1, 5), fontsize=12)  # Show every 5 years

    # Set y-axis ticks appropriately
    valid_ages = combined_df['Earliest Retirement Age'].dropna()
    if not valid_ages.empty:
        min_retirement_age = max(current_age, valid_ages.min() - 1)
        max_retirement_age = valid_ages.max() + 1
        plt.yticks(range(int(min_retirement_age), int(max_retirement_age) + 1, 1), fontsize=12)

    # Add shaded regions for context
    plt.axhspan(current_age, current_age + 10, alpha=0.15, color='#A3BE8C', zorder=0)
    plt.axhspan(current_age + 10, current_age + 20, alpha=0.1, color='#EBCB8B', zorder=0)

    # Add annotations
    plt.text(
        min_age_of_death + 3, current_age + 5, 'Preferred Retirement Zone', fontsize=14, color='#A3BE8C', alpha=0.9
    )

    # Add reference line for current age
    plt.axhline(current_age, color='#BF616A', linestyle='--', alpha=0.7, linewidth=1.5)
    plt.text(
        max_age_of_death - 10,
        current_age + 0.2,
        f'Current Age: {current_age}',
        fontsize=12,
        color='#BF616A',
        ha='right',
        va='bottom',
    )

    # Enhance the legend
    legend = plt.legend(
        title='Scenarios', fontsize=12, title_fontsize=14, frameon=True, loc='upper left', bbox_to_anchor=(1.01, 1)
    )
    legend.get_frame().set_linewidth(2.0)

    # Context information
    plt.figtext(
        0.5,
        0.01,
        income_text[:-2],
        ha='center',
        fontsize=13,
        color='#88C0D0',
        bbox=dict(facecolor='#434C5E', edgecolor='#5E81AC', alpha=0.7, boxstyle='round,pad=0.5'),
    )

    plt.figtext(
        0.5,
        0.04,
        context_text,
        ha='center',
        fontsize=13,
        color='#E5E9F0',
        bbox=dict(facecolor='#434C5E', edgecolor='#5E81AC', alpha=0.7, boxstyle='round,pad=0.5'),
    )

    plt.tight_layout(rect=[0, 0.06, 0.85, 0.97])  # Adjust for legend on the right
    plt.show()

    return combined_df  # Return the data for further analysis if needed

In [ ]:
calculate_retirement_data(
    current_age=38,
    current_yearly_income=120_000,
    current_yearly_expenses=50_500,
    current_savings=0,
    income_increases=[(5, 1.3), (10, 1.3), (15, 1.3)],
    savings_growth_rates=[0.03, 0.05, 0.07],
    inflation_rates=[0.03],
    desired_retirement_expenses=36000,
)

# Enhanced Retirement Calculator with Dark Mode Visualizations

The retirement calculator now includes:

1. **Dark Mode Optimized Design**: Using a Nord-inspired color palette that's easy on the eyes
2. **Seaborn Integration**: Modern statistical visualization with better aesthetics
3. **Visual Context**: Shaded regions showing preferred retirement zones
4. **Annotation Enhancements**: Key information displayed directly on charts
5. **Improved Readability**: Better typography, spacing, and legend placement
6. **Visual Hierarchy**: Important information stands out through strategic use of color and contrast
7. **Two Visualization Views**:
   - Separate subplots for each inflation rate (for detailed comparison)
   - Combined view with all scenarios (for overall pattern recognition)

## Customization Options

The visualization can be further customized with these parameters:

- **Different color palette**: Modify the `nord_palette` variable to use any color scheme
- **Shaded regions**: The shaded areas indicate preferred retirement zones and can be adjusted
- **Income progression**: Multiple income increases at different future time points
- **Growth and inflation scenarios**: Test multiple economic conditions simultaneously
- **Visual style**: Change the Seaborn theme for different visual styles